In [1]:
import numpy as np
import pandas as pd
path_prefix = "/content/drive/MyDrive/Colab Notebooks/learn-pydata/"

# 分组操作

### 初步认识

In [2]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
          "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
          "data1" : np.random.standard_normal(7),
          "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,0.097209,-1.753712
1,a,2,1.091317,-0.818289
2,None,1,0.608637,-1.387382
3,b,2,1.721434,1.434551
4,b,1,0.025603,0.975460
5,a,<NA>,-0.846896,-0.677923
6,None,1,-1.366964,-0.933147


#### 选择某列作为key进行分组

In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [4]:
grouped.mean()

key1
a    0.113877
b    0.873519
Name: data1, dtype: float64

In [5]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1       0.097209
      2       1.091317
b     1       0.025603
      2       1.721434
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,1,2
key1,,
a,0.097209,1.091317
b,0.025603,1.721434


In [8]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.113877,-1.083308
b,1.5,0.873519,1.205006


In [9]:
df.groupby("key2").mean()

,data1,data2
key2,,
1,-0.158879,-0.774695
2,1.406376,0.308131


In [10]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1     0.097209 -1.753712
     2     1.091317 -0.818289
b    1     0.025603  0.975460
     2     1.721434  1.434551

#### 引入外部key分组

In [7]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df["data1"].groupby([states, years]).mean()

CA  2005    0.122211
    2006    0.608637
OH  2005    0.909322
    2006   -0.670680
Name: data1, dtype: float64

#### 查看分组大小

In [11]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [12]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [13]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

#### 分组计数

In [14]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### 在分组上遍历

#### 遍历

In [17]:
for name, group in df.groupby("key1"):
      print("name",name)
      print(group)


name a
  key1  key2     data1     data2
0    a     1  0.097209 -1.753712
1    a     2  1.091317 -0.818289
5    a  <NA> -0.846896 -0.677923
name b
  key1  key2     data1     data2
3    b     2  1.721434  1.434551
4    b     1  0.025603  0.975460


In [20]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
      print((k1, k2))
      print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1  0.097209 -1.753712
('a', 2)
  key1  key2     data1     data2
1    a     2  1.091317 -0.818289
('b', 1)
  key1  key2     data1    data2
4    b     1  0.025603  0.97546
('b', 2)
  key1  key2     data1     data2
3    b     2  1.721434  1.434551


#### 转成字典

In [21]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,1.721434,1.434551
4,b,1,0.025603,0.975460


In [22]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1  0.097209 -1.753712
 1    a     2  1.091317 -0.818289
 5    a  <NA> -0.846896 -0.677923,
 'b':   key1  key2     data1     data2
 3    b     2  1.721434  1.434551
 4    b     1  0.025603  0.975460}

#### 按列分组，遍历

In [23]:
grouped = df.groupby({"key1": "key", "key2": "key",
      "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
       print(group_key)
       print(group_values)

data
      data1     data2
0  0.097209 -1.753712
1  1.091317 -0.818289
2  0.608637 -1.387382
3  1.721434  1.434551
4  0.025603  0.975460
5 -0.846896 -0.677923
6 -1.366964 -0.933147
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


### 选择一个列或列的子集

In [25]:
df.groupby("key1")["data1"]
# 这两种调用作用相同
df["data1"].groupby(df["key1"])

In [26]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1    -1.753712
     2    -0.818289
b    1     0.975460
     2     1.434551

In [29]:
s_grouped = df.groupby(["key1", "key2"])[["data2","data1"]]
s_grouped.mean()

data2     data1
key1 key2                    
a    1    -1.753712  0.097209
     2    -0.818289  1.091317
b    1     0.975460  0.025603
     2     1.434551  1.721434

### 按照字典或者series分组

In [31]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
          columns=["a", "b", "c", "d", "e"],
          index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-1.143984,0.002967,0.568413,0.132780,0.864634
Steve,1.308551,-0.085956,0.106805,1.011776,1.019442
Wanda,0.425276,NaN,NaN,0.686742,-0.716331
Jill,1.831366,-0.591121,-0.107995,0.001562,0.758891
Trey,1.928794,0.686294,0.337257,-0.091261,-1.605923


#### 按字典分组

In [32]:
mapping = {"a": "red", "b": "red", "c": "blue",
            "d": "blue", "e": "red", "f" : "orange"}
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

,blue,red
Joe,0.701194,-0.276383
Steve,1.118581,2.242038
Wanda,0.686742,-0.291056
Jill,-0.106433,1.999137
Trey,0.245997,1.009165


In [33]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

#### 按series分组

In [34]:
people.groupby(map_series, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### 根据传入的函数进行分组

In [35]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.143984,0.002967,0.568413,0.132780,0.864634
4,3.760160,0.095173,0.229262,-0.089698,-0.847031
5,1.733826,-0.085956,0.106805,1.698518,0.303111


In [36]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-1.143984,0.002967,0.568413,0.132780,0.864634
4,two,1.831366,-0.591121,-0.107995,-0.091261,-1.605923
5,one,0.425276,-0.085956,0.106805,0.686742,-0.716331


### 按索引级别分组

In [37]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
        [1, 3, 5, 1, 3]],
        names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.164169 -0.472508 -0.886276 -0.408282  0.334423
1      1.225463 -0.405758 -0.473029  2.167697  0.944801
2     -0.654789 -2.150906 -1.197112  0.943205  0.186818
3      1.802864  1.188772 -0.374166 -0.339263 -0.471148

In [38]:
hier_df.groupby(level="cty", axis="columns").count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [39]:
hier_df.groupby(level="tenor", axis="columns").count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


# 数据聚合

In [40]:
df

,key1,key2,data1,data2
0,a,1,0.097209,-1.753712
1,a,2,1.091317,-0.818289
2,None,1,0.608637,-1.387382
3,b,2,1.721434,1.434551
4,b,1,0.025603,0.975460
5,a,<NA>,-0.846896,-0.677923
6,None,1,-1.366964,-0.933147


### 几种聚合函数

In [42]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     5   -0.846896
      0    0.097209
b     4    0.025603
      3    1.721434
Name: data1, dtype: float64

In [43]:
def peak_to_peak(arr):
        return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.938213,1.075789
b,1,1.695831,0.459091


In [44]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.113877  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.873519  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.594263  1.091317   3.0 -1.083308  0.584814 -1.753712 -1.286001   
b     1.297476  1.721434   2.0  1.205006  0.324626  0.975460  1.090233   

                                    
           50%       75%       max  
key1                                
a    -0.818289 -0.748106 -0.677923  
b     1.205006  1.319778  1.434551  

[2 rows x 24 columns]

### 多功能聚合

In [45]:
tips = pd.read_csv(path_prefix+"examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [46]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [47]:
grouped = tips.groupby(["day", "smoker"])
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

#### 按照指定函数聚合

In [48]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

#### 聚合时传入自定义函数名

In [50]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

#### 每个列多个操作

In [51]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [52]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [53]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

#### 不同的列采取不同操作

In [54]:
grouped.agg({"tip" : np.max, "size" : "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [55]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],"size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 返回没有行索引的聚合数据

In [57]:
tips.groupby(["day", "smoker"]).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [59]:
tips.groupby(["day", "smoker"], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 分组-计算-汇总的一般过程

In [64]:
tips.iloc[232,:]

total_bill      11.61
tip              3.39
smoker             No
day               Sat
time           Dinner
size                2
tip_pct       0.29199
Name: 232, dtype: object

In [60]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [61]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [65]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [67]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [68]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### 不显示分组依据的key

In [71]:
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### 数量和桶分析

In [72]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
        "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,0.530298,-0.222489
1,-0.274547,0.596096
2,-1.107173,0.575858
3,1.110621,-0.460848
4,-1.066020,-0.771729


#### 使用cut等分数据

In [73]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0     (-0.0337, 1.554]
1    (-1.621, -0.0337]
2    (-1.621, -0.0337]
3     (-0.0337, 1.554]
4    (-1.621, -0.0337]
5     (-0.0337, 1.554]
6       (1.554, 3.142]
7     (-3.215, -1.621]
8    (-1.621, -0.0337]
9     (-3.215, -1.621]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.215, -1.621] < (-1.621, -0.0337] < (-0.0337, 1.554] <
                                           (1.554, 3.142]]

In [74]:
def get_stats(group):
      return pd.DataFrame(
      {"min": group.min(), "max": group.max(),
      "count": group.count(), "mean": group.mean()}
      )
grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

min       max  count      mean
data1                                                       
(-3.215, -1.621]  data1 -3.208928 -1.621701     50 -2.071438
                  data2 -2.589463  2.957631     50  0.053686
(-1.621, -0.0337] data1 -1.621007 -0.040966    455 -0.686142
                  data2 -2.724381  3.195451    455 -0.026045
(-0.0337, 1.554]  data1 -0.031769  1.544394    430  0.626647
                  data2 -3.226346  3.226321    430 -0.080577
(1.554, 3.142]    data1  1.565014  3.141520     65  1.992083
                  data2 -2.745433  2.081026     65 -0.024280

In [75]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                        min       max count      mean       min       max   
data1                                                                       
(-3.215, -1.621]  -3.208928 -1.621701    50 -2.071438 -2.589463  2.957631   
(-1.621, -0.0337] -1.621007 -0.040966   455 -0.686142 -2.724381  3.195451   
(-0.0337, 1.554]  -0.031769  1.544394   430  0.626647 -3.226346  3.226321   
(1.554, 3.142]     1.565014  3.141520    65  1.992083 -2.745433  2.081026   

                                   
                  count      mean  
data1                              
(-3.215, -1.621]     50  0.053686  
(-1.621, -0.0337]   455 -0.026045  
(-0.0337, 1.554]    430 -0.080577  
(1.554, 3.142]       65 -0.024280

#### 结合qcut等分数据

In [80]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    2
1    1
2    0
3    3
4    0
Name: data1, dtype: int64

In [81]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.208928 -0.697035    250 -1.288279
      data2 -2.589463  2.957631    250 -0.016001
1     data1 -0.696738 -0.047830    250 -0.373927
      data2 -2.724381  3.195451    250 -0.023209
2     data1 -0.045061  0.666421    250  0.300691
      data2 -3.168659  3.226321    250 -0.084240
3     data1  0.667091  3.141520    250  1.294224
      data2 -3.226346  2.892464    250 -0.058120

### 分组填充缺失的值

In [82]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1    0.638926
2         NaN
3   -0.391158
4         NaN
5   -0.904722
dtype: float64

#### 整体填充同一个值

In [83]:
s.fillna(s.mean())

0   -0.218985
1    0.638926
2   -0.218985
3   -0.391158
4   -0.218985
5   -0.904722
dtype: float64

In [85]:
states = ["Ohio", "New York", "Vermont", "Florida",
    "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
     "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          0.055095
New York      0.672359
Vermont            NaN
Florida      -2.269023
Oregon       -1.597153
Nevada             NaN
California    0.015385
Idaho              NaN
dtype: float64

In [86]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [87]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [88]:
data.groupby(group_key).mean()

East   -0.513856
West   -0.790884
dtype: float64

#### 每组填充不同的值

In [93]:
def fill_mean(group):
      return group.fillna(group.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.055095
New York      0.672359
Vermont      -0.513856
Florida      -2.269023
Oregon       -1.597153
Nevada       -0.790884
California    0.015385
Idaho        -0.790884
dtype: float64

In [94]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
     return group.fillna(fill_values[group.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.055095
New York      0.672359
Vermont       0.500000
Florida      -2.269023
Oregon       -1.597153
Nevada       -1.000000
California    0.015385
Idaho        -1.000000
dtype: float64

### 分组抽样

In [95]:
suits = ["H", "S", "C", "D"] # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [98]:
def draw(deck, n=5):
       return deck.sample(n)
draw(deck)

5H      5
10C    10
7D      7
8S      8
JD     10
dtype: int64

In [100]:
def get_suit(card):
      # last letter is suit
      return card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  AC      1
   9C      9
D  QD     10
   KD     10
H  KH     10
   10H    10
S  5S      5
   QS     10
dtype: int64

In [101]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C     5
KC    10
JD    10
6D     6
QH    10
8H     8
AS     1
6S     6
dtype: int64

### 分组计算加权平均值和方差

In [102]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

,category,data,weights
0,a,0.873923,0.923973
1,a,-0.027513,0.894960
2,a,-1.131402,0.085410
3,a,-1.579603,0.635290
4,b,-0.288640,0.729956
5,b,-0.338684,0.230391
6,b,1.845904,0.524920
7,b,0.396463,0.974595


#### 平均值

In [104]:
grouped = df.groupby("category")
def get_wavg(group):
        return np.average(group["data"], weights=group["weights"])
grouped.apply(get_wavg)

category
a   -0.124932
b    0.433609
dtype: float64

In [105]:
close_px = pd.read_csv(path_prefix+"examples/stock_px.csv", parse_dates=True,
      index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [106]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


#### 计算相关性

In [107]:
def spx_corr(group):
        return group.corrwith(group["SPX"])
rets = close_px.pct_change().dropna()
def get_year(x):
        return x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [108]:
def corr_aapl_msft(group):
        return group["AAPL"].corr(group["MSFT"])
by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 分组线性回归

In [109]:
# 安装命令 conda install statsmodels
import statsmodels.api as sm
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 分组转换 展开组

In [110]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
          'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [119]:
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

#### transform方法

In [120]:
def get_mean(group):
      return group.mean()
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [121]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [122]:
def times_two(group):
      return group * 2
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

#### 分组排序

In [123]:
def get_ranks(group):
      return group.rank(ascending=False)
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

#### 计算均值差/标准差

In [124]:
def normalize(x):
      return (x - x.mean()) / x.std()
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [125]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

#### 组合操作简化上述过程

In [127]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

# 透视表和交叉表

In [128]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


### 透视表

In [129]:
tips.pivot_table(index=["day", "smoker"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

#### 统计平均值

In [130]:
tips.pivot_table(index=["time", "day"], columns="smoker",
      values=["tip_pct", "size"])


size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

#### 统计平均值，带部分汇总

In [131]:
tips.pivot_table(index=["time", "day"], columns="smoker",
      values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

#### 统计条目数

In [132]:
tips.pivot_table(index=["time", "smoker"], columns="day",
      values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

#### 统计时填充数据

In [133]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
      values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### 交叉表

In [135]:
from io import StringIO
data = """Sample Nationality Handedness
        1 USA Right-handed
        2 Japan Left-handed
        3 USA Right-handed
        4 Japan Right-handed
        5 Japan Left-handed
        6 Japan Right-handed
        7 USA Right-handed
        8 USA Left-handed
        9 Japan Right-handed
        10 USA Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [136]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [137]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244